In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import fft
import wave
import sys
import os

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import activations
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras import callbacks
from scipy.io.wavfile import write
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.io import wavfile
import scipy.signal as signal

import torch

In [2]:
path = 'C:/Users/Admin/OneDrive/Skrivebord/Machine Learning/AppliedML2024-main/AppliedML2024/Assignments/Group Project/Shared code/audio/'

sys.path.append("C:/Users/Admin/OneDrive/Skrivebord/Machine Learning/AppliedML2024-main/AppliedML2024/Assignments/Group Project/Shared code/AudioNNRep/functions")
import functions as f

In [3]:
filenames = f.read_files_in_dir(path)
organ = [filename for filename in filenames if "organ" in filename] 
bass = [filename for filename in filenames if "bass" in filename]
guitar = [filename for filename in filenames if "guitar" in filename]
vocal = [filename for filename in filenames if "vocal" in filename] 
flutes = [filename for filename in filenames if "flute" in filename]
keyboards = [filename for filename in filenames if "keyboard" in filename] 

In [4]:
instruments = [organ, bass, guitar, vocal, flutes, keyboards]

In [5]:
import random

In [6]:
#using f.audio_to_waveform(path + instrument) and f.waveform_to_spectogram(waveform_test) generate mixed_spectogram of three of the instruments: pianos, bass, guitar, drum, flutes, keyboards

def generate_mixed_spectrograms(n_mixed_spectrograms, number_of_instruments = 3):
    instruments = [organ, bass, guitar, vocal, flutes, keyboards]
    picked_inst_arr = np.zeros((n_mixed_spectrograms, len(instruments)))
    for i in range(n_mixed_spectrograms):
        # for each row, turn 3 random zeros to 1
        picked_inst = random.sample(range(len(instruments)), number_of_instruments)
        picked_inst_arr[i, picked_inst] = 1
    
    mixed_spectograms = []
    indvidual_spectograms = []
    for i in range(n_mixed_spectrograms):
        selected_files = []
        # Select files from the picked instruments
        for j in range(len(instruments)):
            if picked_inst_arr[i, j] == 1:
                selected_files.append(random.choice(instruments[j]))

        # Generate the mixed spectrogram, and save the individual spectrograms
        spectogram_i = []
        for j in range(len(selected_files)):
            waveform_test, sr = f.audio_to_waveform(path + selected_files[j])
            spectogram = f.waveform_to_spectogram(waveform_test)
            spectogram_i.append(spectogram)
            if j == 0:
                combined_waveform = waveform_test
            else:
                combined_waveform = combined_waveform + waveform_test

        indvidual_spectograms.append(spectogram_i)

        mixed_spectogram = f.waveform_to_spectogram(combined_waveform)
        mixed_spectograms.append(mixed_spectogram)

    return np.array(mixed_spectograms), indvidual_spectograms, picked_inst_arr

In [7]:
mixed_spectograms, indvidual_spectograms, picked_inst_arr = generate_mixed_spectrograms(1000, 3)

In [8]:
np.shape(mixed_spectograms)

(1000, 1025, 126)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, Activation, Input, Concatenate
from tensorflow.keras.models import Model
from torch.utils.data import DataLoader, Dataset

import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F

In [11]:
X_train, X_test, y_train, y_test = train_test_split(mixed_spectograms, picked_inst_arr, test_size=0.2, random_state=42)

# normalize the data
X_train = X_train / np.max(X_train)
X_test  = X_test  / np.max(X_test)

In [13]:
class AudioDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [15]:
train_dataset = AudioDataset(X_train, y_train)
test_dataset = AudioDataset(X_test, y_test)
print(train_dataset)

In [98]:
train_dataset = AudioDataset(X_train, y_train)
test_dataset = AudioDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [99]:
#import device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [100]:
for X, y in train_loader:
    X, y = X.to(device, dtype=torch.float32), y.to(device, dtype=torch.float32)
    X    = torch.unsqueeze(X, 0)
    print("final shape for the model:", np.shape(X))

final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape for the model: torch.Size([1, 32, 1025, 126])
final shape fo

In [101]:
def criterion_function(output, target):
    return torch.sum(torch.abs(target - output))

In [102]:
class CNN(nn.Module):
    def __init__(self, num_classes=6, in_channels=32, out_channels=64):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=(2, 126), padding=(1, 64))
        self.conv2 = nn.Conv2d(out_channels, 32, kernel_size=(3, 3), padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2), stride=2, padding=0)

        # Assuming input size is (batch_size, 32, H, W)
    
        #flatten
        self.fc1 = nn.Linear(8192, 512)
        self.fc2 = nn.Linear(512, in_channels)
        self.fc3 = nn.Linear(in_channels, num_classes)

    def _get_feature_size(self, shape):
        x = torch.zeros(shape)
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.maxpool(x)
        return int(np.prod(x.size()))

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #print("Shape after conv1:", x.shape)
        x = self.maxpool(x)
        #print("Shape after maxpool1:", x.shape)
        x = F.relu(self.conv2(x))
        #print("Shape after conv2:", x.shape)
        x = self.maxpool(x)
        #print("Shape after maxpool2:", x.shape)
        
        x = x.view(x.size(0), -1)
        #print("Shape after flattening again:", x.shape)
        x = F.relu(self.fc1(x))
        #print("Shape after fc1:", x.shape)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
    
        return x

# Initialize model, criterion, optimizer, and other components
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
#criterion = criterion_function()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X, y in train_loader:
        X, y = X.to(device, dtype=torch.float32), y.to(device, dtype=torch.float32)
        
        optimizer.zero_grad()
        y_pred = model(X)
        print(y_pred[0])
        print(y[0])
        loss = criterion_function(y_pred, y)/len(y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X.size(0)  # Accumulate loss
        print(f'Batch loss: {loss.item():.4f}')
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch + 1}, Loss: {epoch_loss:.4f}')

tensor([0.0000, 0.0765, 0.0000, 0.0400, 0.0380, 0.0185],
       grad_fn=<SelectBackward0>)
tensor([1., 0., 1., 0., 0., 1.])
Batch loss: 3.0143
tensor([0.0000, 0.0732, 0.0000, 0.0372, 0.0362, 0.0250],
       grad_fn=<SelectBackward0>)
tensor([1., 0., 1., 0., 1., 0.])
Batch loss: 3.0001
tensor([0.0000, 0.0718, 0.0000, 0.0367, 0.0358, 0.0258],
       grad_fn=<SelectBackward0>)
tensor([1., 0., 1., 1., 0., 0.])
Batch loss: 3.0060
tensor([0.0000, 0.0699, 0.0000, 0.0367, 0.0365, 0.0287],
       grad_fn=<SelectBackward0>)
tensor([0., 1., 0., 0., 1., 1.])
Batch loss: 3.0079
tensor([0.0000, 0.0684, 0.0000, 0.0373, 0.0366, 0.0314],
       grad_fn=<SelectBackward0>)
tensor([1., 1., 0., 0., 0., 1.])
Batch loss: 3.0139
tensor([0.0000, 0.0675, 0.0000, 0.0367, 0.0365, 0.0332],
       grad_fn=<SelectBackward0>)
tensor([1., 0., 1., 1., 0., 0.])
Batch loss: 2.9943
tensor([0.0000, 0.0676, 0.0000, 0.0370, 0.0354, 0.0346],
       grad_fn=<SelectBackward0>)
tensor([0., 1., 0., 1., 0., 1.])
Batch loss: 2.9425